In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_test = pd.read_csv('../input/food-demand-forecasting/test.csv')
df_test

In [ ]:
df_meal = pd.read_csv('../input/food-demand-forecasting/meal_info.csv')
df_center = pd.read_csv('../input/food-demand-forecasting/fulfilment_center_info.csv')
df_meal.head()

In [ ]:
df_center.head()

In [ ]:
df_test = df_test.merge(df_center)
df_test

In [ ]:
df_test = df_test.merge(df_meal)
df_test

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train = pd.read_csv('../input/food-demand-forecasting/train.csv')
df_train

In [ ]:
df_train = df_train.merge(df_center)
df_train

In [ ]:
df_train = df_train.merge(df_meal)
df_train

In [ ]:
df = pd.concat([df_train, df_test])
df

In [ ]:
df_test.columns

In [ ]:
df_train.columns

In [ ]:
df['num_orders'].isnull().sum()

In [ ]:
df.iloc[32569]

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
df['week'] = lb.fit_transform(df['week'])
df['city_code'] = lb.fit_transform(df['city_code'])
df['region_code'] = lb.fit_transform(df['region_code'])
df['op_area'] = lb.fit_transform(df['op_area'])
df['category'] = lb.fit_transform(df['category'])
df['cuisine'] = lb.fit_transform(df['cuisine'])
df['center_type'] = lb.fit_transform(df['center_type'])

In [ ]:
df

In [ ]:
df_id = df['id']
df_id

In [ ]:
df = df.drop(columns = ['id', 'center_id', 'meal_id'], axis=1)


In [ ]:
df

In [ ]:
df_train = df.iloc[0:456548]
df_train

In [ ]:
df_test = df.iloc[456548:]
df_test

In [ ]:
y = df_train['num_orders'].values
y

In [ ]:
X = df_train.drop('num_orders', axis=1)
X

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics 

In [ ]:
RF_pipe = make_pipeline(StandardScaler(),RandomForestRegressor())
RF_pipe.fit(X_train, y_train)
RF_train_y_pred = RF_pipe.predict(X_test)
print(RF_pipe.score(X_test, y_test))
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, RF_train_y_pred)))

In [ ]:
DT_pipe = make_pipeline(StandardScaler(),DecisionTreeRegressor(random_state=1))
DT_pipe.fit(X_train, y_train)
DT_train_y_pred = DT_pipe.predict(X_test)
print(DT_pipe.score(X_test, y_test))
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, DT_train_y_pred)))

In [ ]:
SGD_pipe = make_pipeline(StandardScaler(),SGDRegressor())
SGD_pipe.fit(X_train, y_train)
SGD_train_y_pred = SGD_pipe.predict(X_test)
print(SGD_pipe.score(X_test, y_test))
#print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, SGD_train_y_pred)))

In [ ]:
import xgboost as xgb

XGB_pipe = make_pipeline(StandardScaler(),xgb.XGBRegressor())
XGB_pipe.fit(X_train, y_train)
XGB_train_y_pred = XGB_pipe.predict(X_test)
print(XGB_pipe.score(X_test, y_test))
#print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, XGB_train_y_pred)))

In [ ]:
for keys in RF_pipe.get_params().items():
    print(keys[0],": ",keys[1])

In [ ]:
hyperparameters = {'randomforestregressor__max_features' : ['auto','log2'],
                  'randomforestregressor__max_depth' : [None]}

RF_grid_search = GridSearchCV(RF_pipe, hyperparameters, cv=2, verbose=1)

#Fit and tune model
RF_grid_search.fit(X, y)
print (RF_grid_search.best_params_)
print (RF_grid_search.refit)

In [ ]:
df_test = df_test.drop('num_orders', axis=1)
df_test

In [ ]:
df_test_pred = RF_grid_search.predict(df_test)
df_test_pred

In [ ]:
df_sub = pd.read_csv('../input/food-demand-forecasting/sample_submission.csv')
df_sub

In [ ]:
df_sub['num_orders'] = df_test_pred

In [ ]:
df_sub

In [ ]:
df_sub.to_csv('./submission.csv')

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df_sub = pd.read_csv("../input/demand-forecasting-rmsle-54-96/submission.csv", index_col=0)
df_sub

In [ ]:
df_test = pd.read_csv("../input/food-demand-forecasting/test.csv")
df_test

In [ ]:
df_test['num_orders'] = df_sub['num_orders']

In [ ]:
df_test

In [ ]:
df_train = pd.read_csv("../input/food-demand-forecasting/train.csv")
df_train

In [ ]:
sns.set(rc={'figure.figsize':(70,30)})
sns.set_theme(style="darkgrid")
sns.pointplot(x=df_train['week'], y=df_train['num_orders'])

In [ ]:
sns.set(rc={'figure.figsize':(10,6)})
sns.set_theme(style="darkgrid")
sns.pointplot(x=df_test['week'], y=df_test['num_orders'])

In [ ]:
# for weak 154 mean of number of oreders is:
df_test["num_orders"].loc[df_test["week"]==154].mean()